# Part 4: Buid a Mode Choice Model
## Choose Model and Estimated Parameters
    Build a discrete choice models for at least 2 OD pairs for 2022 data.



In [2]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import logit
import statsmodels.api as sm
import scipy as sp
import geopandas as gpd
from shapely import wkt

In [3]:
#### Store the url string that hosts our .csv files 
#### Copy data files to the 2022CMS folder that should be in the same folder as this notebook
urlday = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Day_2022.csv"
urlhhl = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Household_2022.csv"
urlper = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Person_2022.csv"
urlzon = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Survey_Zones.csv"
urltrp = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Trip_2022.csv"
urlveh = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Vehicle_2022.csv"

#### Read the .csv file using `pandas` libriary and store it as a `pandas` Data Frame
dday = pd.read_csv(urlday)
dhhl = pd.read_csv(urlhhl)
dper = pd.read_csv(urlper)
dzon = pd.read_csv(urlzon)
dtrp = pd.read_csv(urltrp)
dveh = pd.read_csv(urlveh)

#### Finally, let's make sure that all columns are displayed
pd.set_option('display.max_columns', None)

In [4]:
zone_nums = list(range(12))
zone_names = {k: v for k, v in enumerate(dzon['CMS Zone'])}

dtrp['o_cms_zone_name'] = dtrp['o_cms_zone'].map(zone_names)
dtrp['d_cms_zone_name'] = dtrp['d_cms_zone'].map(zone_names)

tripzone_d = dtrp.groupby('d_cms_zone_name').size()
tripzone_o = dtrp.groupby('o_cms_zone_name').size()

all_zones = set(zone_names.values())
o_zones_with_data = set(tripzone_o.index)
d_zones_with_data = set(tripzone_d.index)

missing_origins = all_zones - o_zones_with_data
missing_destinations = all_zones - d_zones_with_data

tripzone_o = tripzone_o.reindex(all_zones, fill_value=0)
tripzone_d = tripzone_d.reindex(all_zones, fill_value=0)

tripzone = pd.crosstab(dtrp['o_cms_zone_name'], dtrp['d_cms_zone_name'])
tripzone = tripzone.reindex(index=all_zones, columns=all_zones, fill_value=0)

In [5]:
dth = pd.merge(dtrp,dhhl, how = 'outer', on='hh_id')
dthp = pd.merge(dth,dper, how = 'outer', on=['hh_id','person_id'])
dthp.describe()

,hh_id,person_id,person_num_x,day_id,travel_dow,day_num,is_complete,linked_trip_id,linked_leg_num,trip_id,trip_num,day_is_complete,arrive_dow,depart_dow,o_in_region,o_county,d_in_region,d_county,mode_type_nyc,r_mode_type_nyc,mode_1,mode_2,mode_3,mode_priority_nyc,transit_access,r_transit_access,transit_egress,r_transit_egress,park_location,park_type,park_pay,park_cost,ev_charge_station,ev_charge_station_level_1,ev_charge_station_level_2,ev_charge_station_level_3,ev_charge_station_level_4,ev_charge_station_level_5,ev_charge_station_level_997,ev_charge_station_level_998,tnc_type,taxi_type,taxi_pay,taxi_cost,transit_type,bike_park_loc,scooter_park_location,num_travelers,num_hh_travelers,num_non_hh_travelers,hh_member_1,hh_member_2,hh_member_3,hh_member_4,hh_member_5,hh_member_6,hh_member_7,hh_member_8,hh_member_9,hh_member_10,hh_member_11,hh_member_12,driver,o_purpose,o_purpose_category,d_purpose,d_purpose_category,d_purpose_category_linked,r_d_purpose_category_linked,work_trip,depart_hour,depart_minute,depart_seconds,r_timeofday,arrive_hour,arrive_minute,arrive_second,duration_minutes,r_duration_minutes,duration_seconds,distance_miles,r_distance_miles,dwell_mins,high_speed_flag,long_distance_flag,o_cms_zone,d_cms_zone,unlinked_trip,o_location_type,d_location_type,trip_wkday_weight,trip_svnday_weight,hh_cms_zone_gp,participation_group,num_days_complete_x,num_days_complete_weekday,num_days_complete_weekend,num_complete_mon,num_complete_tue,num_complete_wed,num_complete_thu,num_complete_fri,num_complete_sat,num_complete_sun,num_people,r_num_people,num_adults,num_kids,num_students,num_workers,num_vehicles,r_num_vehicles,num_trips_x,income_broad,primary_language,residence_type,residence_rent_own,home_county,vehicle_change,reduced_veh_1,reduced_veh_2,reduced_veh_3,reduced_veh_4,reduced_veh_5,reduced_veh_6,reduced_veh_7,reduced_veh_8,reduced_veh_997,increased_veh_1,increased_veh_2,increased_veh_3,increased_veh_4,increased_veh_5,increased_veh_6,increased_veh_7,increased_veh_8,increased_veh_997,ev_typical_charge_1,ev_typical_charge_2,ev_typical_charge_3,ev_typical_charge_4,ev_typical_charge_5,ev_typical_charge_6,ev_typical_charge_7,ev_typical_charge_997,ev_purchase,ev_purchase_barriers_1,ev_purchase_barriers_2,ev_purchase_barriers_3,ev_purchase_barriers_4,ev_purchase_barriers_5,num_bicycles,bicycle_type_1,bicycle_type_2,bicycle_type_997,bike_change,reduced_bike_1,reduced_bike_2,reduced_bike_3,reduced_bike_4,reduced_bike_5,reduced_bike_6,reduced_bike_7,reduced_bike_8,reduced_bike_997,increased_bike_1,increased_bike_2,increased_bike_3,increased_bike_4,increased_bike_5,increased_bike_6,increased_bike_7,increased_bike_8,increased_bike_9,increased_bike_997,bike_store_1,bike_store_2,bike_store_3,bike_store_4,bike_store_5,bike_store_6,bike_store_8,bike_store_9,bike_store_997,num_scooters,scooter_typical_charge_1,scooter_typical_charge_2,scooter_typical_charge_4,scooter_typical_charge_5,scooter_typical_charge_6,scooter_typical_charge_7,scooter_typical_charge_997,packages,hh_weight,num_trips_y,num_days_complete_y,person_num_y,is_participant,phone_type,relationship,age,r_age,race_1,race_2,race_3,race_4,race_5,race_6,race_997,race_999,r_race,ethnicity_1,ethnicity_2,ethnicity_3,ethnicity_4,ethnicity_5,ethnicity_997,ethnicity_999,r_ethnicity,gender,can_drive,disability,disability_follow_1,disability_follow_2,disability_follow_3,disability_follow_4,disability_follow_5,disability_follow_6,disability_follow_996,disability_follow_999,education,employment,worker,num_jobs,job_type,industry,work_in_region,work_county,work_cms_zone,wfh_policy,commute_freq,telework_freq,r_telework_freq,work_mode,r_work_mode,work_mode_own,work_mode_auto,work_park_loc,work_park_pay,work_park_cost,work_park_amount,work_mode_taxi,work_mode_bus,work_mode_rail,work_mode_ferry,work_mode_bike,work_bike_park,work_mode_scooter,work_scooter_park,student,r_student,school_type,r_school_type,school_in_region,school_county,school_freq,remote_class_freq,school_mode,school_mode_own,school_mod

In [6]:
dthp = dthp[dthp['r_mode_type_nyc'] != 995]
mode_name = {1: 'Vehicle', 2: 'Bus', 3: 'Rail', 4: 'Walk', 5: 'Bike', 6: 'Other'}
dthp_mode = dthp.groupby('r_mode_type_nyc').size()
dthp_mode.index = dthp_mode.index.map(mode_name)

dthp_mode

r_mode_type_nyc
Vehicle    23014
Bus         4456
Rail       15944
Walk       36992
Bike        1958
Other       1102
dtype: int64

In [7]:
dthp['slow_speed'] = 0
dthp.loc[:,'slow_speed'] = np.where((dthp['r_mode_type_nyc'] == 4) | (dthp['r_mode_type_nyc'] == 5), 1, 0)
dthp['slow_speed'].value_counts()

slow_speed
0    48746
1    38950
Name: count, dtype: int64

In [8]:
dthp['pub_transit'] = 0
dthp.loc[:,'pub_transit'] = np.where((dthp['r_mode_type_nyc'] == 2) | (dthp['r_mode_type_nyc'] == 3), 1 ,0)
dthp['pub_transit'].value_counts()

pub_transit
0    67296
1    20400
Name: count, dtype: int64

In [9]:
dthp['private'] = 0
dthp.loc[:,'private'] = np.where((dthp['r_mode_type_nyc'] == 1) | (dthp['r_mode_type_nyc'] == 6), 1 ,0)
dthp['private'].value_counts()

private
0    63580
1    24116
Name: count, dtype: int64

In [10]:
dthp['income_lower'] = 0
dthp['education_lower'] = 0
dthp['Age_teenager'] = 0
dthp['Gender_f'] = 0
dthp['HH_size_ge5'] = 0
dthp['driver_license'] = 0
dthp['vehicle_ava'] = 0

dthp.loc[:,'income_lower'] = dthp['income_broad'].map({1: 1, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0})
dthp.loc[:,'education_lower'] = dthp['education'].map({1: 1, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0})
dthp.loc[:,'Age_teenager'] = dthp['age'].map({1: 0, 2: 1, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0})
dthp.loc[:,'Gender_f'] = dthp['gender'].map({1: 1, 2: 0})
dthp.loc[:,'HH_size_ge5'] = dthp['num_people'] >= 5
dthp.loc[:, 'driver_license'] = dthp['can_drive'].map({1: 1, 3: 0})
dthp.loc[:, 'vehicle_ava'] = dthp['num_vehicles'].map({0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1})

dthp['HH_size_ge5'] = dthp['HH_size_ge5'].astype(int)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_24232\3590995467.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.  0.  0. ... nan nan nan]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dthp.loc[:,'income_lower'] = dthp['income_broad'].map({1: 1, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0})
C:\Users\Administrator\AppData\Local\Temp\ipykernel_24232\3590995467.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.  0.  0. ...  0.  0. nan]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dthp.loc[:,'education_lower'] = dthp['education'].map({1: 1, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0})
C:\Users\Administrator\AppData\Local\Temp\ipykernel_24232\3590995467.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future e

In [11]:
data_model_choice = dthp.dropna(subset=['income_lower', 'education_lower','Age_teenager', 'Gender_f',
                               'HH_size_ge5', 'driver_license', 'vehicle_ava'], inplace=False)

In [12]:
data_model_choice.describe()

,hh_id,person_id,person_num_x,day_id,travel_dow,day_num,is_complete,linked_trip_id,linked_leg_num,trip_id,trip_num,day_is_complete,arrive_dow,depart_dow,o_in_region,o_county,d_in_region,d_county,mode_type_nyc,r_mode_type_nyc,mode_1,mode_2,mode_3,mode_priority_nyc,transit_access,r_transit_access,transit_egress,r_transit_egress,park_location,park_type,park_pay,park_cost,ev_charge_station,ev_charge_station_level_1,ev_charge_station_level_2,ev_charge_station_level_3,ev_charge_station_level_4,ev_charge_station_level_5,ev_charge_station_level_997,ev_charge_station_level_998,tnc_type,taxi_type,taxi_pay,taxi_cost,transit_type,bike_park_loc,scooter_park_location,num_travelers,num_hh_travelers,num_non_hh_travelers,hh_member_1,hh_member_2,hh_member_3,hh_member_4,hh_member_5,hh_member_6,hh_member_7,hh_member_8,hh_member_9,hh_member_10,hh_member_11,hh_member_12,driver,o_purpose,o_purpose_category,d_purpose,d_purpose_category,d_purpose_category_linked,r_d_purpose_category_linked,work_trip,depart_hour,depart_minute,depart_seconds,r_timeofday,arrive_hour,arrive_minute,arrive_second,duration_minutes,r_duration_minutes,duration_seconds,distance_miles,r_distance_miles,dwell_mins,high_speed_flag,long_distance_flag,o_cms_zone,d_cms_zone,unlinked_trip,o_location_type,d_location_type,trip_wkday_weight,trip_svnday_weight,hh_cms_zone_gp,participation_group,num_days_complete_x,num_days_complete_weekday,num_days_complete_weekend,num_complete_mon,num_complete_tue,num_complete_wed,num_complete_thu,num_complete_fri,num_complete_sat,num_complete_sun,num_people,r_num_people,num_adults,num_kids,num_students,num_workers,num_vehicles,r_num_vehicles,num_trips_x,income_broad,primary_language,residence_type,residence_rent_own,home_county,vehicle_change,reduced_veh_1,reduced_veh_2,reduced_veh_3,reduced_veh_4,reduced_veh_5,reduced_veh_6,reduced_veh_7,reduced_veh_8,reduced_veh_997,increased_veh_1,increased_veh_2,increased_veh_3,increased_veh_4,increased_veh_5,increased_veh_6,increased_veh_7,increased_veh_8,increased_veh_997,ev_typical_charge_1,ev_typical_charge_2,ev_typical_charge_3,ev_typical_charge_4,ev_typical_charge_5,ev_typical_charge_6,ev_typical_charge_7,ev_typical_charge_997,ev_purchase,ev_purchase_barriers_1,ev_purchase_barriers_2,ev_purchase_barriers_3,ev_purchase_barriers_4,ev_purchase_barriers_5,num_bicycles,bicycle_type_1,bicycle_type_2,bicycle_type_997,bike_change,reduced_bike_1,reduced_bike_2,reduced_bike_3,reduced_bike_4,reduced_bike_5,reduced_bike_6,reduced_bike_7,reduced_bike_8,reduced_bike_997,increased_bike_1,increased_bike_2,increased_bike_3,increased_bike_4,increased_bike_5,increased_bike_6,increased_bike_7,increased_bike_8,increased_bike_9,increased_bike_997,bike_store_1,bike_store_2,bike_store_3,bike_store_4,bike_store_5,bike_store_6,bike_store_8,bike_store_9,bike_store_997,num_scooters,scooter_typical_charge_1,scooter_typical_charge_2,scooter_typical_charge_4,scooter_typical_charge_5,scooter_typical_charge_6,scooter_typical_charge_7,scooter_typical_charge_997,packages,hh_weight,num_trips_y,num_days_complete_y,person_num_y,is_participant,phone_type,relationship,age,r_age,race_1,race_2,race_3,race_4,race_5,race_6,race_997,race_999,r_race,ethnicity_1,ethnicity_2,ethnicity_3,ethnicity_4,ethnicity_5,ethnicity_997,ethnicity_999,r_ethnicity,gender,can_drive,disability,disability_follow_1,disability_follow_2,disability_follow_3,disability_follow_4,disability_follow_5,disability_follow_6,disability_follow_996,disability_follow_999,education,employment,worker,num_jobs,job_type,industry,work_in_region,work_county,work_cms_zone,wfh_policy,commute_freq,telework_freq,r_telework_freq,work_mode,r_work_mode,work_mode_own,work_mode_auto,work_park_loc,work_park_pay,work_park_cost,work_park_amount,work_mode_taxi,work_mode_bus,work_mode_rail,work_mode_ferry,work_mode_bike,work_bike_park,work_mode_scooter,work_scooter_park,student,r_student,school_type,r_school_type,school_in_region,school_county,school_freq,remote_class_freq,school_mode,school_mode_own,school_mod

In [13]:
# choose two specific origin-destination pairs to display (Upper Manhattan to Upper Manhattan, LGA to LGA, and Outer Brooklyn to LGA)
dtrpz8o = data_model_choice[data_model_choice['o_cms_zone_name'] == 'Upper Manhattan']
dtrpz4o = data_model_choice[data_model_choice['o_cms_zone_name'] == 'LGA']
dtrpz3o = data_model_choice[data_model_choice['o_cms_zone_name'] == 'Outer Brooklyn']

dtrpz828 = dtrpz8o[dtrpz8o['d_cms_zone_name'] == 'Upper Manhattan']
dtrpz424 = dtrpz4o[dtrpz4o['d_cms_zone_name'] == 'LGA']
dtrpz324 = dtrpz3o[dtrpz3o['d_cms_zone_name'] == 'LGA']

display(dtrpz828)
display(dtrpz424)
display(dtrpz324)

,hh_id,person_id,person_num_x,day_id,travel_dow,day_num,is_complete,travel_date,linked_trip_id,linked_leg_num,trip_id,trip_num,day_is_complete,arrive_dow,depart_dow,o_in_region,o_county,d_in_region,d_county,mode_type_nyc,r_mode_type_nyc,mode_1,mode_2,mode_3,mode_priority_nyc,transit_access,r_transit_access,transit_egress,r_transit_egress,park_location,park_type,park_pay,park_cost,ev_charge_station,ev_charge_station_level_1,ev_charge_station_level_2,ev_charge_station_level_3,ev_charge_station_level_4,ev_charge_station_level_5,ev_charge_station_level_997,ev_charge_station_level_998,tnc_type,taxi_type,taxi_pay,taxi_cost,transit_type,bike_park_loc,scooter_park_location,num_travelers,num_hh_travelers,num_non_hh_travelers,hh_member_1,hh_member_2,hh_member_3,hh_member_4,hh_member_5,hh_member_6,hh_member_7,hh_member_8,hh_member_9,hh_member_10,hh_member_11,hh_member_12,driver,o_purpose,o_purpose_category,d_purpose,d_purpose_category,d_purpose_category_linked,r_d_purpose_category_linked,work_trip,depart_date,depart_hour,depart_minute,depart_seconds,r_timeofday,arrive_date,arrive_hour,arrive_minute,arrive_second,duration_minutes,r_duration_minutes,duration_seconds,distance_miles,r_distance_miles,dwell_mins,high_speed_flag,long_distance_flag,o_cms_zone,d_cms_zone,unlinked_trip,o_location_type,d_location_type,trip_wkday_weight,trip_svnday_weight,o_cms_zone_name,d_cms_zone_name,hh_cms_zone_gp,participation_group,signup_platform,diary_platform,num_days_complete_x,num_days_complete_weekday,num_days_complete_weekend,num_complete_mon,num_complete_tue,num_complete_wed,num_complete_thu,num_complete_fri,num_complete_sat,num_complete_sun,num_people,r_num_people,num_adults,num_kids,num_students,num_workers,num_vehicles,r_num_vehicles,num_trips_x,income_broad,primary_language,residence_type,residence_rent_own,home_county,vehicle_change,reduced_veh_1,reduced_veh_2,reduced_veh_3,reduced_veh_4,reduced_veh_5,reduced_veh_6,reduced_veh_7,reduced_veh_8,reduced_veh_997,increased_veh_1,increased_veh_2,increased_veh_3,increased_veh_4,increased_veh_5,increased_veh_6,increased_veh_7,increased_veh_8,increased_veh_997,ev_typical_charge_1,ev_typical_charge_2,ev_typical_charge_3,ev_typical_charge_4,ev_typical_charge_5,ev_typical_charge_6,ev_typical_charge_7,ev_typical_charge_997,ev_purchase,ev_purchase_barriers_1,ev_purchase_barriers_2,ev_purchase_barriers_3,ev_purchase_barriers_4,ev_purchase_barriers_5,num_bicycles,bicycle_type_1,bicycle_type_2,bicycle_type_997,bike_change,reduced_bike_1,reduced_bike_2,reduced_bike_3,reduced_bike_4,reduced_bike_5,reduced_bike_6,reduced_bike_7,reduced_bike_8,reduced_bike_997,increased_bike_1,increased_bike_2,increased_bike_3,increased_bike_4,increased_bike_5,increased_bike_6,increased_bike_7,increased_bike_8,increased_bike_9,increased_bike_997,bike_store_1,bike_store_2,bike_store_3,bike_store_4,bike_store_5,bike_store_6,bike_store_8,bike_store_9,bike_store_997,num_scooters,scooter_typical_charge_1,scooter_typical_charge_2,scooter_typical_charge_4,scooter_typical_charge_5,scooter_typical_charge_6,scooter_typical_charge_7,scooter_typical_charge_997,packages,hh_weight,num_trips_y,num_days_complete_y,person_num_y,is_participant,phone_type,relationship,age,r_age,race_1,race_2,race_3,race_4,race_5,race_6,race_997,race_999,r_race,ethnicity_1,ethnicity_2,ethnicity_3,ethnicity_4,ethnicity_5,ethnicity_997,ethnicity_999,r_ethnicity,gender,can_drive,disability,disability_follow_1,disability_follow_2,disability_follow_3,disability_follow_4,disability_follow_5,disability_follow_6,disability_follow_996,disability_follow_999,education,employment,worker,num_jobs,job_type,industry,work_in_region,work_county,work_cms_zone,wfh_policy,commute_freq,telework_freq,r_telework_freq,work_mode,r_work_mode,work_mode_own,work_mode_auto,work_park_loc,work_park_pay,work_park_cost,work_park_amount,work_mode_taxi,work_mode_bus,work_mode_rail,work_mode_ferry,work_mode_bike,work_bike_park,work_mode_scooter,work_scooter_park,student,r_student,school_type,r_school_type,

,hh_id,person_id,person_num_x,day_id,travel_dow,day_num,is_complete,travel_date,linked_trip_id,linked_leg_num,trip_id,trip_num,day_is_complete,arrive_dow,depart_dow,o_in_region,o_county,d_in_region,d_county,mode_type_nyc,r_mode_type_nyc,mode_1,mode_2,mode_3,mode_priority_nyc,transit_access,r_transit_access,transit_egress,r_transit_egress,park_location,park_type,park_pay,park_cost,ev_charge_station,ev_charge_station_level_1,ev_charge_station_level_2,ev_charge_station_level_3,ev_charge_station_level_4,ev_charge_station_level_5,ev_charge_station_level_997,ev_charge_station_level_998,tnc_type,taxi_type,taxi_pay,taxi_cost,transit_type,bike_park_loc,scooter_park_location,num_travelers,num_hh_travelers,num_non_hh_travelers,hh_member_1,hh_member_2,hh_member_3,hh_member_4,hh_member_5,hh_member_6,hh_member_7,hh_member_8,hh_member_9,hh_member_10,hh_member_11,hh_member_12,driver,o_purpose,o_purpose_category,d_purpose,d_purpose_category,d_purpose_category_linked,r_d_purpose_category_linked,work_trip,depart_date,depart_hour,depart_minute,depart_seconds,r_timeofday,arrive_date,arrive_hour,arrive_minute,arrive_second,duration_minutes,r_duration_minutes,duration_seconds,distance_miles,r_distance_miles,dwell_mins,high_speed_flag,long_distance_flag,o_cms_zone,d_cms_zone,unlinked_trip,o_location_type,d_location_type,trip_wkday_weight,trip_svnday_weight,o_cms_zone_name,d_cms_zone_name,hh_cms_zone_gp,participation_group,signup_platform,diary_platform,num_days_complete_x,num_days_complete_weekday,num_days_complete_weekend,num_complete_mon,num_complete_tue,num_complete_wed,num_complete_thu,num_complete_fri,num_complete_sat,num_complete_sun,num_people,r_num_people,num_adults,num_kids,num_students,num_workers,num_vehicles,r_num_vehicles,num_trips_x,income_broad,primary_language,residence_type,residence_rent_own,home_county,vehicle_change,reduced_veh_1,reduced_veh_2,reduced_veh_3,reduced_veh_4,reduced_veh_5,reduced_veh_6,reduced_veh_7,reduced_veh_8,reduced_veh_997,increased_veh_1,increased_veh_2,increased_veh_3,increased_veh_4,increased_veh_5,increased_veh_6,increased_veh_7,increased_veh_8,increased_veh_997,ev_typical_charge_1,ev_typical_charge_2,ev_typical_charge_3,ev_typical_charge_4,ev_typical_charge_5,ev_typical_charge_6,ev_typical_charge_7,ev_typical_charge_997,ev_purchase,ev_purchase_barriers_1,ev_purchase_barriers_2,ev_purchase_barriers_3,ev_purchase_barriers_4,ev_purchase_barriers_5,num_bicycles,bicycle_type_1,bicycle_type_2,bicycle_type_997,bike_change,reduced_bike_1,reduced_bike_2,reduced_bike_3,reduced_bike_4,reduced_bike_5,reduced_bike_6,reduced_bike_7,reduced_bike_8,reduced_bike_997,increased_bike_1,increased_bike_2,increased_bike_3,increased_bike_4,increased_bike_5,increased_bike_6,increased_bike_7,increased_bike_8,increased_bike_9,increased_bike_997,bike_store_1,bike_store_2,bike_store_3,bike_store_4,bike_store_5,bike_store_6,bike_store_8,bike_store_9,bike_store_997,num_scooters,scooter_typical_charge_1,scooter_typical_charge_2,scooter_typical_charge_4,scooter_typical_charge_5,scooter_typical_charge_6,scooter_typical_charge_7,scooter_typical_charge_997,packages,hh_weight,num_trips_y,num_days_complete_y,person_num_y,is_participant,phone_type,relationship,age,r_age,race_1,race_2,race_3,race_4,race_5,race_6,race_997,race_999,r_race,ethnicity_1,ethnicity_2,ethnicity_3,ethnicity_4,ethnicity_5,ethnicity_997,ethnicity_999,r_ethnicity,gender,can_drive,disability,disability_follow_1,disability_follow_2,disability_follow_3,disability_follow_4,disability_follow_5,disability_follow_6,disability_follow_996,disability_follow_999,education,employment,worker,num_jobs,job_type,industry,work_in_region,work_county,work_cms_zone,wfh_policy,commute_freq,telework_freq,r_telework_freq,work_mode,r_work_mode,work_mode_own,work_mode_auto,work_park_loc,work_park_pay,work_park_cost,work_park_amount,work_mode_taxi,work_mode_bus,work_mode_rail,work_mode_ferry,work_mode_bike,work_bike_park,work_mode_scooter,work_scooter_park,student,r_student,school_type,r_school_type,

,hh_id,person_id,person_num_x,day_id,travel_dow,day_num,is_complete,travel_date,linked_trip_id,linked_leg_num,trip_id,trip_num,day_is_complete,arrive_dow,depart_dow,o_in_region,o_county,d_in_region,d_county,mode_type_nyc,r_mode_type_nyc,mode_1,mode_2,mode_3,mode_priority_nyc,transit_access,r_transit_access,transit_egress,r_transit_egress,park_location,park_type,park_pay,park_cost,ev_charge_station,ev_charge_station_level_1,ev_charge_station_level_2,ev_charge_station_level_3,ev_charge_station_level_4,ev_charge_station_level_5,ev_charge_station_level_997,ev_charge_station_level_998,tnc_type,taxi_type,taxi_pay,taxi_cost,transit_type,bike_park_loc,scooter_park_location,num_travelers,num_hh_travelers,num_non_hh_travelers,hh_member_1,hh_member_2,hh_member_3,hh_member_4,hh_member_5,hh_member_6,hh_member_7,hh_member_8,hh_member_9,hh_member_10,hh_member_11,hh_member_12,driver,o_purpose,o_purpose_category,d_purpose,d_purpose_category,d_purpose_category_linked,r_d_purpose_category_linked,work_trip,depart_date,depart_hour,depart_minute,depart_seconds,r_timeofday,arrive_date,arrive_hour,arrive_minute,arrive_second,duration_minutes,r_duration_minutes,duration_seconds,distance_miles,r_distance_miles,dwell_mins,high_speed_flag,long_distance_flag,o_cms_zone,d_cms_zone,unlinked_trip,o_location_type,d_location_type,trip_wkday_weight,trip_svnday_weight,o_cms_zone_name,d_cms_zone_name,hh_cms_zone_gp,participation_group,signup_platform,diary_platform,num_days_complete_x,num_days_complete_weekday,num_days_complete_weekend,num_complete_mon,num_complete_tue,num_complete_wed,num_complete_thu,num_complete_fri,num_complete_sat,num_complete_sun,num_people,r_num_people,num_adults,num_kids,num_students,num_workers,num_vehicles,r_num_vehicles,num_trips_x,income_broad,primary_language,residence_type,residence_rent_own,home_county,vehicle_change,reduced_veh_1,reduced_veh_2,reduced_veh_3,reduced_veh_4,reduced_veh_5,reduced_veh_6,reduced_veh_7,reduced_veh_8,reduced_veh_997,increased_veh_1,increased_veh_2,increased_veh_3,increased_veh_4,increased_veh_5,increased_veh_6,increased_veh_7,increased_veh_8,increased_veh_997,ev_typical_charge_1,ev_typical_charge_2,ev_typical_charge_3,ev_typical_charge_4,ev_typical_charge_5,ev_typical_charge_6,ev_typical_charge_7,ev_typical_charge_997,ev_purchase,ev_purchase_barriers_1,ev_purchase_barriers_2,ev_purchase_barriers_3,ev_purchase_barriers_4,ev_purchase_barriers_5,num_bicycles,bicycle_type_1,bicycle_type_2,bicycle_type_997,bike_change,reduced_bike_1,reduced_bike_2,reduced_bike_3,reduced_bike_4,reduced_bike_5,reduced_bike_6,reduced_bike_7,reduced_bike_8,reduced_bike_997,increased_bike_1,increased_bike_2,increased_bike_3,increased_bike_4,increased_bike_5,increased_bike_6,increased_bike_7,increased_bike_8,increased_bike_9,increased_bike_997,bike_store_1,bike_store_2,bike_store_3,bike_store_4,bike_store_5,bike_store_6,bike_store_8,bike_store_9,bike_store_997,num_scooters,scooter_typical_charge_1,scooter_typical_charge_2,scooter_typical_charge_4,scooter_typical_charge_5,scooter_typical_charge_6,scooter_typical_charge_7,scooter_typical_charge_997,packages,hh_weight,num_trips_y,num_days_complete_y,person_num_y,is_participant,phone_type,relationship,age,r_age,race_1,race_2,race_3,race_4,race_5,race_6,race_997,race_999,r_race,ethnicity_1,ethnicity_2,ethnicity_3,ethnicity_4,ethnicity_5,ethnicity_997,ethnicity_999,r_ethnicity,gender,can_drive,disability,disability_follow_1,disability_follow_2,disability_follow_3,disability_follow_4,disability_follow_5,disability_follow_6,disability_follow_996,disability_follow_999,education,employment,worker,num_jobs,job_type,industry,work_in_region,work_county,work_cms_zone,wfh_policy,commute_freq,telework_freq,r_telework_freq,work_mode,r_work_mode,work_mode_own,work_mode_auto,work_park_loc,work_park_pay,work_park_cost,work_park_amount,work_mode_taxi,work_mode_bus,work_mode_rail,work_mode_ferry,work_mode_bike,work_bike_park,work_mode_scooter,work_scooter_park,student,r_student,school_type,r_school_type,

## Discuss Resulted Model Validity
    Only reasonable models should be included in the report. To increase the likelihood to generate the model with a good fit, use OD-pairs with largest number of predicted trips. 

In [14]:
# slow_speed model choice for trips from Upper Manhattan to Upper Manhattan

logit_model_slow_z828 = sm.Logit.from_formula("slow_speed ~ 1 + income_lower + HH_size_ge5 + education_lower + Gender_f + driver_license + vehicle_ava", data=dtrpz828)
result_slow_z828 = logit_model_slow_z828.fit()
print(result_slow_z828.summary())

Optimization terminated successfully.
         Current function value: 0.574984
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:             slow_speed   No. Observations:                 7381
Model:                          Logit   Df Residuals:                     7374
Method:                           MLE   Df Model:                            6
Date:                Wed, 19 Nov 2025   Pseudo R-squ.:                 0.02290
Time:                        17:57:48   Log-Likelihood:                -4244.0
converged:                       True   LL-Null:                       -4343.4
Covariance Type:            nonrobust   LLR p-value:                 3.179e-40
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           1.3851      0.082     16.929      0.000       1.225       1.545
income_lower  

In [15]:
# slow_speed model choice for trips from LGA to LGA

logit_model_slow_z424 = sm.Logit.from_formula("slow_speed ~ 1 + income_lower + HH_size_ge5 + education_lower + Gender_f + driver_license + vehicle_ava", data=dtrpz424)
result_slow_z424 = logit_model_slow_z424.fit()
print(result_slow_z424.summary())

Optimization terminated successfully.
         Current function value: 0.615704
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:             slow_speed   No. Observations:                12988
Model:                          Logit   Df Residuals:                    12981
Method:                           MLE   Df Model:                            6
Date:                Wed, 19 Nov 2025   Pseudo R-squ.:                0.003967
Time:                        17:57:48   Log-Likelihood:                -7996.8
converged:                       True   LL-Null:                       -8028.6
Covariance Type:            nonrobust   LLR p-value:                 7.976e-12
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.8973      0.061     14.827      0.000       0.779       1.016
income_lower  

In [16]:
# slow_speed model choice for trips from Outer Brooklyn to LGA

logit_model_slow_z324 = sm.Logit.from_formula("slow_speed ~ 1 + income_lower + HH_size_ge5 + education_lower + Gender_f + driver_license + vehicle_ava", data=dtrpz324)
result_slow_z324 = logit_model_slow_z324.fit()
print(result_slow_z324.summary())

Optimization terminated successfully.
         Current function value: 0.447320
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             slow_speed   No. Observations:                 1221
Model:                          Logit   Df Residuals:                     1214
Method:                           MLE   Df Model:                            6
Date:                Wed, 19 Nov 2025   Pseudo R-squ.:                 0.02551
Time:                        17:57:48   Log-Likelihood:                -546.18
converged:                       True   LL-Null:                       -560.47
Covariance Type:            nonrobust   LLR p-value:                 7.267e-05
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.9960      0.205     -4.865      0.000      -1.397      -0.595
income_lower  

In terms of slow-speed model, all models are converged and reasonable based on the signs and significance of relative coefficients. The number of iteration for 3 models is 5 or 6, which is a reasonable number. Although the Pseudo R-squared values are relatively small, they distribute within a certain range. The reason is the individual heterogeneity will affect the travel survey data. But, the sign and coefficients remain stable and reasonable, indicating that the models can catch the relationships among variables. In conclusion, the 3 models are statistically valid and produce interpretable outcomes.

In [17]:
# public transit model choice for trips from LGA to Upper Manhattan
logit_model_pubtransit_z828 = sm.Logit.from_formula("pub_transit ~ 1 + income_lower + HH_size_ge5 + education_lower + Gender_f + driver_license + vehicle_ava", data=dtrpz828)
result_pubtransit_z828 = logit_model_pubtransit_z828.fit()
print(result_pubtransit_z828.summary())

Optimization terminated successfully.
         Current function value: 0.445485
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:            pub_transit   No. Observations:                 7381
Model:                          Logit   Df Residuals:                     7374
Method:                           MLE   Df Model:                            6
Date:                Wed, 19 Nov 2025   Pseudo R-squ.:                 0.02338
Time:                        17:57:48   Log-Likelihood:                -3288.1
converged:                       True   LL-Null:                       -3366.8
Covariance Type:            nonrobust   LLR p-value:                 2.058e-31
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.3094      0.090    -14.539      0.000      -1.486      -1.133
income_lower  

In [18]:
# public transit model choice for trips from LGA to LGA
logit_model_pubtransit_z424 = sm.Logit.from_formula("pub_transit ~ 1 + income_lower + HH_size_ge5 + education_lower + Gender_f + driver_license + vehicle_ava", data=dtrpz424)
result_pubtransit_z424 = logit_model_pubtransit_z424.fit()
print(result_pubtransit_z424.summary())

Optimization terminated successfully.
         Current function value: 0.572527
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:            pub_transit   No. Observations:                12988
Model:                          Logit   Df Residuals:                    12981
Method:                           MLE   Df Model:                            6
Date:                Wed, 19 Nov 2025   Pseudo R-squ.:                0.001083
Time:                        17:57:48   Log-Likelihood:                -7436.0
converged:                       True   LL-Null:                       -7444.0
Covariance Type:            nonrobust   LLR p-value:                   0.01314
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.1216      0.064    -17.621      0.000      -1.246      -0.997
income_lower  

In [19]:
# public transit model choice for trips from Outer Brooklyn to LGA
logit_model_pubtransit_z324 = sm.Logit.from_formula("pub_transit ~ 1 + income_lower + HH_size_ge5 + education_lower + Gender_f + driver_license + vehicle_ava", data=dtrpz324)
result_pubtransit_z324 = logit_model_pubtransit_z324.fit()
print(result_pubtransit_z324.summary())

Optimization terminated successfully.
         Current function value: 0.562701
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:            pub_transit   No. Observations:                 1221
Model:                          Logit   Df Residuals:                     1214
Method:                           MLE   Df Model:                            6
Date:                Wed, 19 Nov 2025   Pseudo R-squ.:                 0.03703
Time:                        17:57:48   Log-Likelihood:                -687.06
converged:                       True   LL-Null:                       -713.48
Covariance Type:            nonrobust   LLR p-value:                 1.263e-09
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.4636      0.181      2.563      0.010       0.109       0.818
income_lower  

Overall, the 3 public-transit models demonstrate acceptable validity. All models successfully converged within a small number of iterations, indicating stable numerical performance. The pseudo R-squared remain within the expected range for mode-choice and travel-survey data. Despite differences in sample sizes, the models present consistent coefficient patterns, suggesting that they are capturing stable behavioral relationships rather than random noise. In general, the tests for all three models are statistically significant, confirming that the models perform better than the null specification. In conclusion, the 3 models are statistically valid and produce interpretable outcomes.


In [21]:
# private vehicle model choice for trips from LGA to Upper Manhattan
logit_model_private_z828 = sm.Logit.from_formula("private ~ 1 + income_lower + HH_size_ge5 + education_lower + Gender_f + driver_license + vehicle_ava", data=dtrpz828)
result_private_z828 = logit_model_private_z828.fit()
print(result_private_z828.summary())

Optimization terminated successfully.
         Current function value: 0.282667
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                private   No. Observations:                 7381
Model:                          Logit   Df Residuals:                     7374
Method:                           MLE   Df Model:                            6
Date:                Wed, 19 Nov 2025   Pseudo R-squ.:                  0.1593
Time:                        17:58:12   Log-Likelihood:                -2086.4
converged:                       True   LL-Null:                       -2481.6
Covariance Type:            nonrobust   LLR p-value:                1.678e-167
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -4.3544      0.171    -25.436      0.000      -4.690      -4.019
income_lower  

In [ ]:
# private vehicle model choice for trips from LGA to LGA
logit_model_private_z424 = sm.Logit.from_formula("private ~ 1 + income_lower + HH_size_ge5 + education_lower + Gender_f + driver_license + vehicle_ava", data=dtrpz424)
result_private_z424 = logit_model_private_z424.fit()
print(result_private_z424.summary())

Optimization terminated successfully.
         Current function value: 0.176555
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                private   No. Observations:                12988
Model:                          Logit   Df Residuals:                    12981
Method:                           MLE   Df Model:                            6
Date:                Wed, 19 Nov 2025   Pseudo R-squ.:                 0.09357
Time:                        15:22:48   Log-Likelihood:                -2293.1
converged:                       True   LL-Null:                       -2529.8
Covariance Type:            nonrobust   LLR p-value:                 4.482e-99
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -3.2164      0.138    -23.361      0.000      -3.486      -2.947
income_lower  

In [ ]:
# private vehicle model choice for trips from Outer Brooklyn to LGA
logit_model_private_z324 = sm.Logit.from_formula("private ~ 1 + income_lower + HH_size_ge5 + education_lower + Gender_f + driver_license + vehicle_ava", data=dtrpz324)
result_private_z324 = logit_model_private_z324.fit()
print(result_private_z324.summary())

Optimization terminated successfully.
         Current function value: 0.291620
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                private   No. Observations:                 1221
Model:                          Logit   Df Residuals:                     1214
Method:                           MLE   Df Model:                            6
Date:                Wed, 19 Nov 2025   Pseudo R-squ.:                 0.09743
Time:                        15:22:48   Log-Likelihood:                -356.07
converged:                       True   LL-Null:                       -394.50
Covariance Type:            nonrobust   LLR p-value:                 1.580e-14
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -2.0654      0.278     -7.420      0.000      -2.611      -1.520
income_lower  

Across the 3 private-mode models, the estimation results indicate that the specifications are statistically valid. All models reached convergence within 7–8 iterations, which reflects stable likelihood maximization. The pseudo R-squared values, ranging roughly from 0.09 to 0.16, are higher than those of typical mode-choice models using household travel survey data, suggesting that the explanatory variables capture a substantial portion of the behavioral variation. In addition, the signs and magnitudes of coefficients remain consistent across the different datasets, implying that the behavioral relationships are stable. Overall, the private-mode models demonstrate strong statistical validity and reliable performance.

## Mode Choice Model Results Discussions
    Discuss R-squired, intercept value, signs and significance of estimated parameters. 
    Discuss multiple noticeable observations based on estimated parameters: what unexpected significant factors you have found? Is there a difference in the models for different evaluated OD-pairs? 


**Slow speed**

1. Upper Manhattan → Upper Manhattan (z828)
    - R-squared

    The pseudo R-squared of 0.0229 is low but reasonable for a logit model based on heterogenous travel survey data. This value suggests limited explanatory power from the observed socioeconomic variables but still indicates improved fit compared to the null model.

    - Intercept value

    The intercept is high and positive (1.3851, p<0.001). This implies that slow-speed conditions have a high baseline application in Upper Manhattan before considering any variables. This is consistent with the dense and highly congested nature of this area.

    - Coefficient Signs and Significance

    `income_lower` (–0.3165, p<0.001) indicates lower-income travelers experience lower possibility of slow-speed, likely due to higher reliance on walking/transit modes not captured fully in speed distribution. The p value < 0.001. There is statistical evidence to conclude that lower income has a relationship with slow speed mode.

    `education_lower` (–0.2092, p=0.027) is significant but modest in magnitude. The p value is 0.027 < 0.5. This means we can reject the null hypothesis. There is statistical evidence to conclude that lower education has a relationship with slow speed mode.

    `Gender_f` (–0.2704, p<0.001) suggests female travelers face slightly lower odds of slow-speed. The p value < 0.001. There is statistical evidence to conclude that female has a relationship with slow speed mode.

    `vehicle_ava` (–0.6460, p<0.001) is the strongest predictor, indicating vehicle access strongly reduces slow-speed risk. The p value < 0.001. There is statistical evidence to conclude that household having vehicles has a relationship with slow speed mode.

    **Notable Observations:**

    `driver_license` and `household size` are not significant, which is reasonable because trips often involve transit or slow speed travel, weakening the influence of license possession or household structure.

2. LGA → LGA (z424)
    - R-squared
    
    The pseudo R-squared of 0.00397 is very low but not unusual for a logit model based on heterogeneous travel survey data.

    - Intercept value
    
    The intercept is positive and statistically significant (0.8973, p<0.001). This suggests that slow-speed conditions have a relatively high baseline likelihood in the full sample before considering any explanatory variables.

    - Coefficient Signs and Significance

    `income_lower (0.0312, p=0.564)` This coefficient is not statistically significant. There is no statistical evidence to conclude that lower income has a relationship with slow-speed mode.

    `HH_size_ge5 (0.0595, p=0.550)`
    The coefficient is not significant. Larger household size does not show a statistical association with slow-speed conditions, which is reasonable since most trips are determined by individual behavior rather than household structure.

    `education_lower (–0.3169, p=0.001)`
    This variable is significant and negative. With p=0.001 < 0.05, there is statistical evidence to conclude that lower education level is associated with a lower likelihood of slow-speed mode. This may reflect differences in mode choice or travel patterns correlated with education.

    `Gender_f (0.0980, p=0.011)`
    The coefficient is positive and statistically significant. The p-value < 0.05 provides evidence that gender has a relationship with slow-speed mode, although the effect size is modest.

    `driver_license (–0.0540, p=0.342)`
    This variable is not significant. Possession of a driver’s license does not statistically influence whether a traveler is categorized into slow-speed mode, which is reasonable given that many trips do not require private vehicle operation.

    `vehicle_ava (–0.2487, p<0.001)`
    This is the strongest predictor in the model. The negative and highly significant coefficient indicates that vehicle availability substantially reduces the likelihood of slow-speed mode. The p-value < 0.001 provides strong statistical evidence of this relationship.

    **Notable Observations:**

    Vehicle availability remains the most influential factor affecting travel speed category. Education level and gender also show statistically significant effects. In contrast, income, household size, and driver’s license status show no significant relationships, suggesting that these socioeconomic characteristics contribute minimally to explaining slow-speed travel model choice.


3. Outer Brooklyn → LGA (z324)

    - R-squared
    
    The pseudo R-squared of 0.0255 is low but typical for logit models using heterogeneous travel survey data.

    - Intercept value
    
    The intercept is negative and statistically significant (–0.9960, p<0.001). This indicates that the baseline likelihood of slow-speed mode in this zone is relatively low before accounting for any socioeconomic factors. Because this OD-pair trip is long-distance cross-area travel, where travelers are unlike to choose slow speed modes, such as walking and bicycles.

    - Coefficient Signs and Significance

    `income_lower (0.4240, p=0.016)`
    The coefficient is positive and statistically significant. The p-value < 0.05 provides statistical evidence that lower-income travelers have a higher likelihood of using the slow-speed category. This may indicate stronger reliance on walking or transit modes among lower-income households.

    `HH_size_ge5 (–0.2047, p=0.609)`
    This variable is not significant. Large household size does not show any statistical relationship with slow-speed mode in this zone.

    `education_lower (–0.1663, p=0.665)`
    Not statistically significant. There is no evidence that education level influences slow-speed travel in this model.

    `Gender_f (–0.5286, p=0.001)`
    The coefficient is negative and significant. The p-value < 0.05 provides evidence that female travelers are less likely to exhibit slow-speed behavior. This may reflect differences in mode choice or trip purpose patterns within the zone.

    `driver_license (–0.3849, p=0.040)`
    Statistically significant. Travelers with a driver’s license have lower odds of slow-speed mode, which is consistent with higher likelihood of using faster modes such as private vehicles or ride services.

    `vehicle_ava (–0.5169, p=0.014)`
    A significant and negative coefficient. Vehicle availability strongly reduces the probability of slow-speed mode. The p-value < 0.05 indicates a clear statistical relationship between having access to a vehicle and faster travel speeds.

    **Notable Observations:**

    income, gender, and driver’s license status show statistically significant relationships with slow-speed travel. Vehicle availability remains an important predictor as well. Education level and household size do not contribute meaningfully to explaining speed outcomes in this zone, and the overall explanatory power of the model remains modest.



**Public transit**

1. Upper Manhattan → Upper Manhattan (z828)
    - R-squared

    The pseudo R-squared is 0.0234, which is low but reasonable for a logit model using heterogeneous travel survey data.

    - Intercept value

    The intercept is negative and significant (–1.3094, p<0.001), suggesting that, before considering other variables, the baseline probability of choosing public transit for these trips is relatively low. 

    - Coefficient Signs and Significance

    `income_lower (0.1206, p=0.129)` is positive but not statistically significant. Lower-income travelers may have slightly higher public transit usage, but there is insufficient evidence to conclude a strong effect.

    `HH_size_ge5 (0.0292, p=0.831)` is not significant, suggesting household size has negligible impact on cross-regional public transit choice.

    `education_lower (–0.0478, p=0.680)` is also not significant, indicating that lower education does not meaningfully influence transit usage in this context.

    `Gender_f (0.3064, p<0.001)` is positive and significant, meaning female travelers are more likely to use public transit for trips.

    `driver_license (–0.4043, p<0.001)` is negative and significant, showing that possession of a driver’s license reduces the likelihood of choosing public transit.

    `vehicle_ava (–0.5395, p<0.001)` is the strongest negative predictor, indicating that household vehicle availability significantly decreases public transit usage.

    **Notable Observations**

    The signs and coefficients suggest that in these travel trips have a generally low probability of using public transit, consistent with the practical expectation that longer-distance travel is more likely to involve faster modes (e.g., personal vehicles). Gender and vehicle availability are the most influential factors for public transit choice in this scenario.

2. LGA → LGA (z424)
    - R-squared
    
    The pseudo R-squared of 0.0011 is very low but not unusual for a logit model based on heterogeneous travel survey data.

    - Intercept value
    
    The intercept is positive and statistically significant (–1.1216, p<0.001). This reflects a low baseline probability of choosing public transit in this area.

    - Coefficient Signs and Significance

    `income_lower (–0.0411, p=0.467)`
    Not significant. Income level does not show a meaningful relationship with public transit usage in this zone.

    `HH_size_ge5 (–0.1601, p=0.147)`
    Not significant. Larger households are not more or less inclined toward transit in this dataset.

    `education_lower (–0.0603, p=0.574)`
    Also not significant, indicating no clear influence of education level on transit choice.

    `Gender_f (0.0367, p=0.365)`
    Not significant. Female travelers show no statistically distinguishable difference in public transit use.

    `driver_license (0.1307, p=0.029)`
    This coefficient is positive and statistically significant. This is somewhat ***counterintuitive***, because driver’s license is usually associated with lower transit usage. This result may reflect the characteristics of this zone, where licensed individuals still depend on transit due to congestion, limited parking, or transit-friendly commuting patterns.

    `vehicle_ava (–0.1102, p=0.012)`
    Significant and negative. Households with available vehicles have lower odds of using public transit, consistent with expectations and typical travel behavior patterns.

    **Notable Observations**

    The model shows very weak explanatory power, and most socioeconomic factors are not significant. Only driver’s license possession and vehicle availability show statistical relationships with transit choice. This suggests that for this zone, public transit usage depends far more on functional, spatial, or network-level factors than household characteristics.


3. Outer Brooklyn → LGA (z324)

    - R-squared
    
    The pseudo R-squared of 0.0370 is small but typical for logit models using heterogeneous travel survey data.

    - Intercept value
    
    The intercept is positive and significant (0.4636, p=0.010). This indicates a relatively high baseline likelihood of choosing public transit even before considering individual characteristics. For cross-regional trips, it aligns with the fact that public transit options (subway + commuter rail + express bus) are widely used and competitive.

    - Coefficient Signs and Significance

    `income_lower (–0.5012, p=0.001)`
    Significant and negative. Lower-income travelers are ***less likely*** to take public transit for these longer trips, which is the opposite of the usual pattern. This likely reflects the characteristics of cross-region travel: many such trips involve long-distance commuting to higher-employment districts where low-income individuals rely more on fast speed (e.g. vehicles) modes or do not make these long-distance trips as frequently.

    `HH_size_ge5 (–0.2478, p=0.405)`
    Not significant. Larger households show no clear tendency toward or against public transit for these trips.

    `education_lower (–0.0146, p=0.963)`
    Completely insignificant. Education does not influence mode choice here.

    `Gender_f (0.5355, p<0.001)`
    Significant and positive. Female travelers are more likely to use public transit for this type of long-distance movement.

    `driver_license (0.6762, p<0.001)`
    Strongly positive and significant. This is a striking result but a logical one in this scenario. With cross-regional travel, many drivers still rely on transit because driving across zones is expensive, stressful, and slow. In NYC, having a license does not imply frequent car use, as many licensed adults never drive regularly.

    `vehicle_ava (–0.4945, p=0.002)`
    Significant and negative. Households with vehicles remain less likely to use public transit, consistent with expectations, though the magnitude is moderate rather than extreme.

    **Notable Observations:**

    This model captures the complexity of cross-region travel. The positive coefficients for both gender and driver’s license possession show that female and drivers are more likely to choose public tranist. The negative coefficient for lower income household shows distinctive result.


**Private transit**

1. Upper Manhattan → Upper Manhattan (z828)
    - R-squared

    The pseudo R-squared of 0.0229 is low but reasonable for a logit model based on heterogenous travel survey data. This value suggests limited explanatory power from the observed socioeconomic variables but still indicates improved fit compared to the null model.

    - Intercept value

    The intercept is negative and significant (–4.3544, p<0.001). This indicates that most individuals would not attempt to choose private travel as default. In dense urban districts, private mode is never the optimal mode as the expensive cost for parking and severe congestion.

    - Coefficient Signs and Significance

    `income_lower (0.6071, p<0.001)`
    Highly significant and positive. Lower-income individuals are more likely to rely on private vehicles. This may reflect a spatial pattern: lower-income households living in areas with weaker transit coverage may still depend on low-cost used cars for daily travel.

    `HH_size_ge5 (–0.2930, p=0.053)`
    Weakly significant and negative. Larger households show a slight tendency against private mode. Because private car can be used across many people in a family, resulting extra cost and scheduling conflicts.

    `education_lower (0.6268, p<0.001)`
    Significant and positive. Lower-education groups are more likely to travel by private vehicle. Education often proxies for occupation type and residential location; many lower-education households live in transit-poor environments where car dependence is high.

    `Gender_f (0.1409, p=0.089)`
    Marginally insignificant. Gender doesn’t really affect whether people choose private modes.

    `driver_license (0.7763, p<0.001)`
    Strongly significant and positive. This makes sense: holding a driver’s license definitely boosts the odds of choosing private mode. It more reflects capability rather than preference.

    `vehicle_ava (2.2162, p<0.001)`
    The largest coefficient in the model. Vehicle availability overwhelmingly predicts private travel. Without a vehicle, private mode is generally impossible; with one, the probability rises sharply. This variable directively defines the feasibility of private travel.

    **Notable Observations**

    This model behaves exactly as expected for urban travel. The strong effects of license possession and vehicle availability show that access dominates over general demographics. The high pseudo R-squared highlights how cleanly private travel behavior follows these structural drivers.

2. LGA → LGA (z424)
    - R-squared
    
    The pseudo R-squared of 0.0936 is lower than earlier model (0.1593) but still substantial for cross-regional travel.

    - Intercept value
    
    The intercept is negative and significant (–3.2164, p<0.001). This suggests that most travelers would not choose private mode for trips.

    - Coefficient Signs and Significance

    `income_lower (0.1497, p=0.228)`
    Positive but not significant. Lower-income households may slightly favor private mode, but the effect is not statistically evidence to stand for it.

    `HH_size_ge5 (0.2369, p=0.153)`
    Positive but not significant. Larger households show a small tendency toward private travel, but coordination costs and scheduling friction likely counteract this effect.

    `education_lower (1.0203, p<0.001)`
    Strongly positive and significant. Lower-education households are more likely to travel by private vehicle.

    `Gender_f (–0.6875, p<0.001)`
    Significant and negative. Female travelers are less likely to choose private mode in cross-regional trips, possibly due to risk perception or travel responsibilities.

    `driver_license (–0.4825, p<0.001)`
    Significant and negative. This may reflect that short, local trips are often made by walking, biking, or transit, so having a license does not strongly increase private travel for these short-distance trips.

    `vehicle_ava (1.6154, p<0.001)`
    Strongly positive. Without a vehicle, private mode is largely infeasible


3. Outer Brooklyn → LGA (z324)

    - R-squared
    
    The pseudo R-squared is 0.0974, which is reasonable for a cross-regional logit model.

    - Intercept value
    
    The intercept is negative and significant (–2.0654, p<0.001), suggesting a low baseline probability of choosing private mode for these cross-regional trips.

    - Coefficient Signs and Significance

    `income_lower (0.4282, p=0.082)`
    Positive and marginally non-significant. Lower-income travelers may have a slightly higher tendency to use private vehicles, but the effect is weak.

    `HH_size_ge5 (0.6413, p=0.074)`
    Positive and marginally significant. Larger households may show a mild tendency toward private travel, potentially for group coordination, but this is not strong statistically.

    `education_lower (0.2523, p=0.531)`
    Not significant. Education level does not meaningfully influence private-mode choice for these cross-regional trips.

    `Gender_f (–0.3497, p=0.100)`
    Negative but not significant. Female travelers may be slightly less likely to choose private mode.
    `driver_license (–0.9663, p<0.001)`
    Strongly negative and significant. Surprisingly, having a driver’s license reduces private-mode choice for cross-regional trips. This likely reflects coordination difficulties, scheduling conflicts, and costs associated with multi-person or long-distance travel.

    `vehicle_ava (1.6370, p<0.001)`
    Strongly positive and highly significant. Household vehicle availability is the main enabler of private-mode travel. Without access to a vehicle, private travel is largely impossible.

    **Notable Observations**

    For Outer Brooklyn → LGA trips, structural factors like vehicle availability dominate private travel decisions, while individual characteristics (income, household size, gender) have weaker influence. The strong negative effect of having a driver’s license likely reflects the frictions of coordinating long-distance trips. Overall, private-mode choice is highly dependent on vehicle access, while cross-regional constraints suppress the impact of other socioeconomic variables.